# 🏢 Vendor Risk Scorer

## ATLAS Capital Delivery - Subcontractor Performance Model

This notebook builds an XGBoost model to score vendor/subcontractor risk:
- Analyzes change order history, on-time performance, quality metrics
- Compares against trade category benchmarks
- Provides composite risk score (0-100)
- SHAP explanations for procurement decisions

**Business Value**: Identify risky subcontractors before contract award.

In [ ]:
# Imports
from snowflake.snowpark import Session
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error, r2_score
import shap

# Connect to Snowflake
connection_params = {"connection_name": "demo"}
session = Session.builder.configs(connection_params).create()
session.use_database("CAPITAL_PROJECTS_DB")
session.use_warehouse("CAPITAL_ML_WH")
print(f"Connected: {session.get_current_account()}")

In [ ]:
# Load vendor data with CO aggregates
vendors_df = session.table("ATOMIC.VENDOR").to_pandas()

# Get CO statistics per vendor
co_stats = session.sql("""
    SELECT 
        VENDOR_ID,
        COUNT(*) as co_count,
        SUM(APPROVED_AMOUNT) as co_total,
        AVG(APPROVED_AMOUNT) as co_avg,
        COUNT(DISTINCT PROJECT_ID) as project_count,
        SUM(CASE WHEN ML_CATEGORY = 'SCOPE_GAP' THEN 1 ELSE 0 END) as scope_gap_count,
        SUM(CASE WHEN APPROVAL_LEVEL = 'AUTO' THEN 1 ELSE 0 END) as auto_approved_count
    FROM ATOMIC.CHANGE_ORDER
    WHERE STATUS = 'APPROVED'
    GROUP BY VENDOR_ID
""").to_pandas()

# Merge
df = vendors_df.merge(co_stats, on='VENDOR_ID', how='left').fillna(0)
print(f"Vendors loaded: {len(df)}")

In [ ]:
# Feature engineering
df['CO_RATE'] = df['co_count'] / (df['project_count'] + 1)  # COs per project
df['SCOPE_GAP_RATIO'] = df['scope_gap_count'] / (df['co_count'] + 1)
df['AUTO_APPROVE_RATIO'] = df['auto_approved_count'] / (df['co_count'] + 1)

feature_cols = ['AVG_CO_RATE', 'ONTIME_DELIVERY_RATE', 'QUALITY_SCORE', 
                'CO_RATE', 'SCOPE_GAP_RATIO', 'AUTO_APPROVE_RATIO', 'co_total']

# Fill missing values
for col in feature_cols:
    if col in df.columns:
        df[col] = df[col].fillna(df[col].median() if df[col].median() > 0 else 50)

# Target: existing risk score (or could train on historical outcomes)
X = df[feature_cols].values
y = df['RISK_SCORE'].values

print(f"Features: {feature_cols}")
print(f"Risk score range: {y.min():.0f} - {y.max():.0f}")

In [ ]:
# Train XGBoost model
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = XGBRegressor(n_estimators=100, max_depth=4, learning_rate=0.1, random_state=42)
model.fit(X_train, y_train)

# Evaluate
y_pred = model.predict(X_test)
print(f"R² Score: {r2_score(y_test, y_pred):.3f}")
print(f"MAE: {mean_absolute_error(y_test, y_pred):.1f} points")

# SHAP analysis
explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(X_test)

importance_df = pd.DataFrame({
    'feature': feature_cols,
    'importance': np.abs(shap_values).mean(0)
}).sort_values('importance', ascending=False)

print("\\n📊 Top Risk Factors:")
for _, row in importance_df.iterrows():
    print(f"  • {row['feature']}: {row['importance']:.2f}")

In [ ]:
# Generate risk scores for all vendors
all_scores = model.predict(X).clip(0, 100)

# Create output dataframe
score_df = df[['VENDOR_ID']].copy()
score_df['RISK_SCORE'] = all_scores.astype(int)
score_df['RISK_TIER'] = pd.cut(all_scores, bins=[0, 30, 50, 70, 100], 
                                labels=['LOW', 'MEDIUM', 'HIGH', 'CRITICAL'])
score_df['SCORE_DATE'] = pd.Timestamp.now().date()
score_df['MODEL_NAME'] = 'VENDOR_RISK_SCORER'
score_df['MODEL_VERSION'] = '1.0'

# Add component scores
score_df['CO_RATE_SCORE'] = (df['CO_RATE'] / df['CO_RATE'].max() * 100).fillna(50).astype(int)
score_df['ONTIME_SCORE'] = ((1 - df['ONTIME_DELIVERY_RATE']) * 100).fillna(50).astype(int)
score_df['QUALITY_SCORE'] = (100 - df['QUALITY_SCORE']).fillna(50).astype(int)

# Save to Snowflake
sp_scores = session.create_dataframe(score_df)
sp_scores.write.mode('overwrite').save_as_table('ML.VENDOR_RISK_SCORES')

print("\\n🏢 Vendor Risk Summary:")
print(score_df['RISK_TIER'].value_counts())
print(f"\\nHighest risk vendor: {df.loc[all_scores.argmax(), 'VENDOR_NAME']}")
print("\\n✅ Risk scores saved to ML.VENDOR_RISK_SCORES")

In [ ]:
# Identify Apex Electrical - the key vendor in Hidden Discovery
apex = df[df['VENDOR_NAME'] == 'Apex Electrical Services']
if len(apex) > 0:
    apex_row = apex.iloc[0]
    print("🔍 APEX ELECTRICAL ANALYSIS")
    print("=" * 40)
    print(f"Risk Score: {apex_row['RISK_SCORE']:.0f}/100")
    print(f"CO Count: {apex_row['co_count']:.0f}")
    print(f"Scope Gap COs: {apex_row['scope_gap_count']:.0f}")
    print(f"Total CO Amount: ${apex_row['co_total']:,.0f}")
    print(f"On-Time Rate: {apex_row['ONTIME_DELIVERY_RATE']*100:.0f}%")
    print("\\n⚠️ This vendor is central to the Hidden Discovery pattern!")

session.close()
print("\\n✅ Notebook complete!")